<a href="https://colab.research.google.com/github/AinaRB/2022_TempReconstructionSaoPaulo_EO_ML/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define environment

In [178]:
#Check whether we are driving in the drive or locally
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

We're running Colab


In [195]:
!git clone git@github.com:AinaRB/2022_TempReconstructionSaoPaulo_EO_ML.git

Cloning into '2022_TempReconstructionSaoPaulo_EO_ML'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [180]:

# insert your desired path to work on
import os
from os.path import join
project_path = "S:\Projects\AinaRB_ClimateHealth\Part2_Spatial\Obj2_TempModelling\scripts\ESA_internship\2022_TempReconstructionSaoPaulo_EO_ML"
project_path

'S:\\Projects\\AinaRB_ClimateHealth\\Part2_Spatial\\Obj2_TempModelling\\scripts\\ESA_internship\x822_TempReconstructionSaoPaulo_EO_ML'

In [182]:
#Link code folder to python execution path
import sys
sys.path.append(project_path)

In [ ]:
# Detect changes in external code and to automatically update it without restarting the runtime.
%load_ext autoreload
%autoreload 2

In [183]:
#Define plotting settings
import matplotlib
font = {'family':'Arial', 'size':'15', 'weight':'normal'}

matplotlib.rc('font', **font)

In [ ]:
# Define folder structure 
config = {
    'main': 'Brazil',
    'output': join(project_path, "code", "models"),
    'metrics': join(project_path, "code", "metrics")
}

# List comprehension for the folder structure code
[os.makedirs(val, exist_ok=True) for key, val in config.items()]

In [10]:
#Import libraries
#import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from glob import glob
#from config import DEP_NAMES, GROUPED_VARS, DATA_REDUCER_SETTINGS, DATA_PROCESSING_SETTINGS

In [184]:
 os. getcwd()

'/content'

# Import data and inspect

Since we are working with Google Collab, we first need to import the libraries to browse the file we want to import from our local directory. Once that is selected, then we can read it as a panda dataframe. 

In [196]:
#Import the data
from google.colab import files
uploaded = files.upload()


Saving datasetHandler.py to datasetHandler.py


In [41]:
import io
df=pd.read_csv(io.BytesIO(uploaded['data2018_modelinput.csv']))

Below, we explore the dataframe

In [42]:
df.head()

,station_code,date,lat,lon,source,year,month,day.x,temp_mean75,doy,...,elevation_500m,slope_500m,ImprSuf_SP,dist_to_coast,ESArivers,popdens,ndvi,bsa,sza,lst
0,1000840,2018-01-01,-23.63,-46.58,cgs_stations,2018,1,1,24.326035,1,...,743.772278,0.684851,2,45.233009,1.509531,10224.246094,0.173255,141.360397,1668,29.140158
1,1000840,2018-01-02,-23.63,-46.58,cgs_stations,2018,1,2,24.526826,2,...,743.772278,0.684851,2,45.233009,1.509531,10224.246094,0.173255,141.360397,1668,29.160706
2,1000840,2018-01-03,-23.63,-46.58,cgs_stations,2018,1,3,21.867257,3,...,743.772278,0.684851,2,45.233009,1.509531,10224.246094,0.173255,141.360397,1668,28.434814
3,1000840,2018-01-04,-23.63,-46.58,cgs_stations,2018,1,4,21.616583,4,...,743.772278,0.684851,2,45.233009,1.509531,10224.246094,0.173255,141.360397,1668,28.685966
4,1000840,2018-01-05,-23.63,-46.58,cgs_stations,2018,1,5,21.909021,5,...,743.772278,0.684851,2,45.233009,1.509531,10224.246094,0.173255,141.360397,1668,28.808559


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18615 entries, 0 to 18614
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   station_code    18615 non-null  object 
 1   date            18615 non-null  object 
 2   lat             18615 non-null  float64
 3   lon             18615 non-null  float64
 4   source          18615 non-null  object 
 5   year            18615 non-null  int64  
 6   month           18615 non-null  int64  
 7   day.x           18615 non-null  int64  
 8   temp_mean75     18316 non-null  float64
 9   doy             18615 non-null  int64  
 10  daylength       18615 non-null  float64
 11  X               18615 non-null  int64  
 12  day.y           18615 non-null  int64  
 13  elevation_500m  18615 non-null  float64
 14  slope_500m      18615 non-null  float64
 15  ImprSuf_SP      18615 non-null  int64  
 16  dist_to_coast   18615 non-null  float64
 17  ESArivers       18615 non-null 

In [82]:
#Clean dataset
df_c=df.dropna()
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16978 entries, 0 to 18614
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   station_code    16978 non-null  object 
 1   date            16978 non-null  object 
 2   lat             16978 non-null  float64
 3   lon             16978 non-null  float64
 4   source          16978 non-null  object 
 5   year            16978 non-null  int64  
 6   month           16978 non-null  int64  
 7   day.x           16978 non-null  int64  
 8   temp_mean75     16978 non-null  float64
 9   doy             16978 non-null  int64  
 10  daylength       16978 non-null  float64
 11  X               16978 non-null  int64  
 12  day.y           16978 non-null  int64  
 13  elevation_500m  16978 non-null  float64
 14  slope_500m      16978 non-null  float64
 15  ImprSuf_SP      16978 non-null  int64  
 16  dist_to_coast   16978 non-null  float64
 17  ESArivers       16978 non-null 

# Data dictionary

Below we have grouped the data in different relevant groups.

In [127]:
VARS = {
    'TEMP DATA':[
        'temp_mean75'
    ],
    'ATMOSPHERIC DATA': [
       # 'bsa',
        'sza',
        'lst'
    ],
     'GEO DATA': [
        'popdens',
        'ESArivers',
        'slope_500m',
        'elevation_500m',
        'ImprSuf_SP',
        'dist_to_coast'
    ],
    'TEMPORAL DATA': [
        'daylength'
    ]
}

# Data reduction and normalization

In [1]:
#Import libraries
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
import numpy as np


In [67]:
#Define functions for reducing the data:
def pca_reducer(values, ncomponents ):
    normalized = MinMaxScaler().fit_transform(values) #This normalizes the data
    pca = PCA(n_components = ncomponents) #This defines data reduction
    pca.fit(normalized.T) #This performs data reduction

    return  MinMaxScaler().fit_transform(pca.components_.T)

def pls_reducer(values, y, ncomponents):
    normalized_values = MinMaxScaler().fit_transform(values) #This normalizes the data
    X = normalized_values
    pls = PLSRegression(n_components=ncomponents)
    pls_score = pls.fit_transform(X, y)

    return MinMaxScaler().fit_transform(pls_score[0])

*Note: based on this it’s better to set it based on the percentage we are aiming to explain and let it select the actual number of components by itself. The suggested amount is 0.95. Source:https:\\www.mikulskibartosz.name\pca-how-to-choose-the-number-of-components\

 

The reduction and normalisation is done (a) for the response variable, in this case temperature, and (b) for all the prediction variables. The reason why we do it separetly is because the function we have defined above for the reduction embeds already the normalization. 

## a. Normalization for temperature variable (y)
Here we only normalize the data, no reduction is needed as we only have one variable.

In [128]:
#Extract mean temperature from dataframe and apply normalization 
temp=df_c[VARS['TEMP DATA']].values
scaler=MinMaxScaler()
temp_data=scaler.fit_transform(temp)

## b. Normalization + reduction for predictor variables
For now, we will use the PLS reduction as this not only considers the predictors but also the exposure variabel (temperature in our case). 

Important to remember that the n_components value should be in [1, n_features]. For example, for the atmospheric data we have 2 covariates or predictors, and so the range is [1,2] --> n_components=2

In [124]:
# We apply the funciton we defined above and we save the results by groups
atmospheric_data= pls_reducer(df_c[VARS['ATMOSPHERIC DATA']].values, temp_data,len(VARS['ATMOSPHERIC DATA']))
geo_data=pls_reducer(df_c[VARS['GEO DATA']].values, temp_data, len(VARS['GEO DATA']))
temporal_data=pls_reducer(df_c[VARS['TEMPORAL DATA']].values, temp_data, len(VARS['TEMPORAL DATA']))

# Data preparation

### 1) Create a dataframe with the reduced/normalized data
The normalized/reduced data we have generated above is saved as an array. Here call the array elements to save them as columns in a new pd dataframe. 


In [273]:
basic= {'year': df_c['year'], 
        'month': df_c['month'], 
        'day': df_c['day.x'], 
        'date': df_c['date'],
        'dep_id':df_c['station_code']}
atmospheric = {VARS['ATMOSPHERIC DATA'][0]:atmospheric_data[:,0], 
               VARS['ATMOSPHERIC DATA'][1]:atmospheric_data[:,1]}
geo ={VARS['GEO DATA'][0]:geo_data[:,0], 
          VARS['GEO DATA'][1]:geo_data[:,1], 
          VARS['GEO DATA'][2]:geo_data[:,2], 
          VARS['GEO DATA'][3]:geo_data[:,3],
          VARS['GEO DATA'][4]:geo_data[:,4]}
temporal = {VARS['TEMPORAL DATA'][0]:temporal_data[:,0]}
temp = {VARS['TEMP DATA'][0]:temp_data[:,0]}

columns={**basic, **atmospheric, **geo, **temporal, **temp}
red_df=pd.DataFrame(columns)
red_df.head()

,year,month,day,date,dep_id,sza,lst,popdens,ESArivers,slope_500m,elevation_500m,ImprSuf_SP,daylength,temp_mean75
0,2018,1,1,2018-01-01,1000840,0.851713,0.525190,0.799255,0.715779,0.266084,0.45874,0.478603,0.990767,0.647265
1,2018,1,2,2018-01-02,1000840,0.852412,0.526175,0.799255,0.715779,0.266084,0.45874,0.478603,0.988802,0.655606
2,2018,1,3,2018-01-03,1000840,0.827707,0.491383,0.799255,0.715779,0.266084,0.45874,0.478603,0.986654,0.545117
3,2018,1,4,2018-01-04,1000840,0.836255,0.503421,0.799255,0.715779,0.266084,0.45874,0.478603,0.984323,0.534703
4,2018,1,5,2018-01-05,1000840,0.840427,0.509297,0.799255,0.715779,0.266084,0.45874,0.478603,0.981813,0.546852


In [207]:
len(df_c)

16978

### 2) Create training and validation dataframes

The entire underlying premise of ML is to find an algorithm that can explain as well as possible the underlying relationships between our covariates and response variable in order to produce predictions with the minimum possible error. For us to be able to test the performance of the model, we need a validation dataset. 

There are several ways to split the data in training and validation sets. A basic one we can start with for time series is to set aside the last couple of years of our time series or a 30% data. As a starting point, here we will use the last 100 observations (approximately 30% of 365 days)

Useful resources:
 > Numpy array: https://jalammar.github.io/visual-numpy/

In [215]:
#Define the date corersponding to the -30% data
from datetime import datetime
from datetime import timedelta
# Subtract 10 days from a given datetime object
dtObj = datetime.strptime(max(red_df['date']), '%Y-%m-%d')
n = round(365*0.3)
past_date = dtObj - timedelta(days=n)
past_date

datetime.datetime(2018, 9, 12, 0, 0)

In [224]:
#Create training and validation datasets
training_df = red_df[pd.to_datetime(red_df['date']) <= past_date].drop('date', axis=1)
validation_df = red_df[pd.to_datetime(red_df['date']) > past_date].drop('date', axis=1)

ATTENTION: In Alessandro's code there is an overlaping time (both datasets contain 2016). Is this done on purpose?

ATTENTION 2: As we only use one year, we are loosing information on the seasonality by cutting 3 months from the time series. This will likely affect the learning and validaton. I shoudl be using 2 years minimum. 

In [274]:
training_df.head()

,year,month,day,dep_id,sza,lst,popdens,ESArivers,slope_500m,elevation_500m,ImprSuf_SP,daylength,temp_mean75
0,2018,1,1,1000840,0.851713,0.525190,0.799255,0.715779,0.266084,0.45874,0.478603,0.990767,0.647265
1,2018,1,2,1000840,0.852412,0.526175,0.799255,0.715779,0.266084,0.45874,0.478603,0.988802,0.655606
2,2018,1,3,1000840,0.827707,0.491383,0.799255,0.715779,0.266084,0.45874,0.478603,0.986654,0.545117
3,2018,1,4,1000840,0.836255,0.503421,0.799255,0.715779,0.266084,0.45874,0.478603,0.984323,0.534703
4,2018,1,5,1000840,0.840427,0.509297,0.799255,0.715779,0.266084,0.45874,0.478603,0.981813,0.546852


In [275]:
validation_df.head()

,year,month,day,dep_id,sza,lst,popdens,ESArivers,slope_500m,elevation_500m,ImprSuf_SP,daylength,temp_mean75
255,2018,9,13,1000840,0.619422,0.753333,0.799255,0.715779,0.266084,0.45874,0.478603,0.413024,0.380361
256,2018,9,14,1000840,0.634232,0.633018,0.799255,0.715779,0.266084,0.45874,0.478603,0.420690,0.330264
257,2018,9,15,1000840,0.628489,0.624929,0.799255,0.715779,0.266084,0.45874,0.478603,0.428374,0.382199
258,2018,9,16,1000840,0.614395,0.605080,0.799255,0.715779,0.266084,0.45874,0.478603,0.436073,0.330029
259,2018,9,17,1000840,0.583596,0.561705,0.799255,0.715779,0.266084,0.45874,0.478603,0.443788,0.339856


Here we imported into Google Colab the datasetHandler.py file that Alessandro and Raquel created. This object will handle all the operations needed to create, reshape and augment the training and validation dataset to fit the requirements of each Deep Learning or Machine Learning model.

In [237]:
from datasetHandler import datasetHandler
dataset_handler = datasetHandler(training_df, validation_df)

Transform training and validation datasets into vectors using the get data function from within the datasetHandler.py

get_data(t_window_size, t_prediction)
where t_window_size refers to 
and t_prediction refers to 

In [279]:
# Rename to keep similar format as Alessandro
training = training_df
validation = validation_df

In [300]:
#define the moving window for data agumentation (they used 12 months for their monthly dataset...)
t_window_size=12
t_prediction=1

In [358]:
#Get first department from TRAINING DATASET and check data dimensions
stations = pd.unique(training.dep_id)
db_dep = training[training.dep_id == stations[0]]
len_series = db_dep.shape[0] - (t_window_size)
print("Length series: ", len_series, "Stations number: ",len(stations))

# Create an empty x and y for the array with the corret dimensions
## np.zeros() --> Return a new array of given shape and type, filled with zeros. 
## The content is: number or rows,samples,  number of columns)
x_train = np.zeros((len_series*len(stations), t_window_size, training.shape[1]))
y_train = np.zeros((len_series*len(stations), t_prediction))
print('X Training shape', x_train.shape) #This is now a numpy array
print('Y Training shape', y_train.shape) #This is now a numpy array


Length series:  237 Stations number:  51
X Training shape (12087, 12, 13)
Y Training shape (12087, 1)


In [359]:
x_train[0:7,0:5,0:3]

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [326]:
y_train[0:4,...]

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [355]:
data[0:2,1:]

array([[1, 1, '1000840', 0.8517129099406928, 0.5251900800384318,
        0.7992547885160707, 0.7157789287330905, 0.26608406717999555,
        0.45874005170314447, 0.47860318785422173, 0.9907671963300897,
        0.6472646431955067],
       [1, 2, '1000840', 0.8524122301326326, 0.5261749427186495,
        0.7992547885160707, 0.7157789287330905, 0.26608406717999555,
        0.45874005170314447, 0.47860318785422173, 0.9888022345613867,
        0.6556063417690419]], dtype=object)

In [373]:
test=x_train[0, ...] = data[225 : (225)+12, :]
test

ValueError: ignored

In [349]:
 test=y_train[0, ..., 1:] = data[(0 + 12) : (0 + 12 + 1), -1]
 test

array([0.6647345800496136], dtype=object)

In [357]:
data

array([[2018, 1, 15, '1000840', 0.8950295413527044, 0.5861935148057074,
        0.7992547885160707, 0.7157789287330905, 0.26608406717999555,
        0.45874005170314447, 0.47860318785422173, 0.947368452715449,
        0.6647345800496136]], dtype=object)

In [381]:
# Fill the training set
counter = 0
#for count, sta in enumerate(stations):
    db_dep = training[training.dep_id == "1000840"]
    data = db_dep.to_numpy()
    data
    print('\rProcessing departments {} of {}'.format(count, len(stations)), end='\t\t')
   
    for count2 in range(len_series):
                x_train[counter, ...] = data[count2 : (count2)+t_window_size, :]
               # y_train[counter, ..., :t_prediction] = data[(count2 + t_window_size) : (count2 + t_window_size + t_prediction), -2]
               # y_train[counter, ..., t_prediction:] = data[(count2 + t_window_size) : (count2 + t_window_size + t_prediction), -1]
                counter+1



IndentationError: ignored

In [392]:
db_dep = training[training.dep_id == "1000840"]
data = db_dep.to_numpy()
print('\rProcessing stations {} of {}'.format(count, len(stations)), end='\t\t')
   
for count2 in range(len_series):
    x_train[counter, ...] = data[count2 : (count2)+t_window_size,]
               # y_train[counter, ..., :t_prediction] = data[(count2 + t_window_size) : (count2 + t_window_size + t_prediction), -2]
               # y_train[counter, ..., t_prediction:] = data[(count2 + t_window_size) : (count2 + t_window_size + t_prediction), -1]
    counter+1



Processing stations 4 of 51		

In [397]:
 test=x_train[counter, ...] = data[12:12+12,]
 test

array([[2018, 1, 15, '1000840', 0.8950295413527044, 0.5861935148057074,
        0.7992547885160707, 0.7157789287330905, 0.26608406717999555,
        0.45874005170314447, 0.47860318785422173, 0.947368452715449,
        0.6647345800496136],
       [2018, 1, 16, '1000840', 0.907282347767812, 0.603449318234891,
        0.7992547885160707, 0.7157789287330905, 0.26608406717999555,
        0.45874005170314447, 0.47860318785422173, 0.9430511234867092,
        0.6474207218746362],
       [2018, 1, 17, '1000840', 0.8509450309892796, 0.6490024953492642,
        0.7992547885160707, 0.7157789287330905, 0.26608406717999555,
        0.45874005170314447, 0.47860318785422173, 0.9385874453982063,
        0.6536104964784648],
       [2018, 1, 18, '1000840', 0.860255992813406, 0.6621152567469841,
        0.7992547885160707, 0.7157789287330905, 0.26608406717999555,
        0.45874005170314447, 0.47860318785422173, 0.9339807211734054,
        0.7446006159052798],
       [2018, 1, 19, '1000840', 0.8556294311

In [388]:
x_train

array([[[2.01800000e+03, 8.00000000e+00, 3.10000000e+01, ...,
         4.78603188e-01, 3.15472648e-01, 5.92757122e-01],
        [2.01800000e+03, 9.00000000e+00, 1.00000000e+00, ...,
         4.78603188e-01, 3.22805075e-01, 5.27130797e-01],
        [2.01800000e+03, 9.00000000e+00, 2.00000000e+00, ...,
         4.78603188e-01, 3.30170914e-01, 5.08989608e-01],
        ...,
        [2.01800000e+03, 9.00000000e+00, 9.00000000e+00, ...,
         4.78603188e-01, 3.82554676e-01, 3.09891491e-01],
        [2.01800000e+03, 9.00000000e+00, 1.00000000e+01, ...,
         4.78603188e-01, 3.90140298e-01, 3.39502876e-01],
        [2.01800000e+03, 9.00000000e+00, 1.10000000e+01, ...,
         4.78603188e-01, 3.97747813e-01, 3.24794088e-01]],

       [[2.01800000e+03, 1.00000000e+00, 2.00000000e+00, ...,
         4.78603188e-01, 9.88802235e-01, 6.55606342e-01],
        [2.01800000e+03, 1.00000000e+00, 3.00000000e+00, ...,
         4.78603188e-01, 9.86653660e-01, 5.45117062e-01],
        [2.01800000e+03, 

In [380]:
db_dep = training[training.dep_id == "1000840"]
db_dep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 254
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            249 non-null    int64  
 1   month           249 non-null    int64  
 2   day             249 non-null    int64  
 3   dep_id          249 non-null    object 
 4   sza             249 non-null    float64
 5   lst             249 non-null    float64
 6   popdens         249 non-null    float64
 7   ESArivers       249 non-null    float64
 8   slope_500m      249 non-null    float64
 9   elevation_500m  249 non-null    float64
 10  ImprSuf_SP      249 non-null    float64
 11  daylength       249 non-null    float64
 12  temp_mean75     249 non-null    float64
dtypes: float64(9), int64(3), object(1)
memory usage: 27.2+ KB


In [ ]:

            print('\rProcessing departments {} of {}'.format(count, len(stations)), end='\t\t')

            for count2 in range(len_series):
                x_train[counter, ...] = data[count2 : (count2)+t_window_size, :] #takes row position =count2 and copies it all the way to count2+window (i.e. moving window)
                y_train[counter, ..., :t_prediction] = data[(count2 + t_window_size) : (count2 + t_window_size + t_prediction), -2]
                y_train[counter, ..., t_prediction:] = data[(count2 + t_window_size) : (count2 + t_window_size + t_prediction), -1]
                counter+=1

In [ ]:
# Get the first department from the VALIDATION DATASET  to asses data dimensions
stations = pd.unique(validation.dep_id)
db_dep = validation[validation.dep_id == stations[0]]
# len_series  = db_dep.shape[0] - (t_window_size + t_prediction)
len_series = db_dep.shape[0] - (t_window_size)
        
# Initialize validation dataset
x_val = np.zeros((len_series*len(stations), t_window_size, validation.shape[1]))
y_val = np.zeros((len_series*len(stations), 2*t_prediction))
print('\nX Validation shape', x_val.shape)
print('Y Validation shape', y_val.shape)

counter = 0
# Fill the validation set
for count, dep in enumerate(stations):
    db_dep = validation[validation.dep_id == dep]
    data = db_dep.to_numpy()

    print('\rProcessing stations {} of {}'.format(count, len(stations)), end='\t\t')

    for count2 in range(len_series):
      x_val[counter, ...] = data[count2 : (count2)+t_window_size, :]
      y_val[counter, ..., :t_prediction] = data[count2 + t_window_size:count2 + t_window_size + t_prediction, -2]
      y_val[counter, ..., t_prediction:] = data[count2 + t_window_size:count2 + t_window_size + t_prediction, -1]
      counter += 1

 x_val, y_val